In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Obesity/GSE181339'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Study of the usefulness of human peripheral blood mononuclear cells for the analysis of metabolic recovery after weight loss (METAHEALTH-TEST)"
!Series_summary	"The aim of this study is to design and validate a test, METAHEALTH-TEST, based on gene expression analysis in blood cells, to quickly and easily analyse metabolic health. This test will be used to analyse metabolic improvement in overweight/obese individuals and in metabolically obese normal-weight (MONW) individuals after undergoing a weight loss intervention and/or an intervention for improvement in eating habits and lifestyle. Obesity and its medical complications are a serious health problem today. Using peripheral blood mononuclear cells (PBMC) as an easily obtainable source of transcriptomic biomarkers would allow to deepen into the knowledge of adaptations in response to increased adiposity that occur in internal homeostatic tissues, without the need of using invasive biopsies. More

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data
is_gene_available = True

# Identify the relevant rows for 'Obesity', 'age', and 'gender'
trait_row = 1  # OW/OB is related to obesity
age_row = 2
gender_row = 0

# Define conversion functions
def convert_trait(value):
    # Extract the value after the colon
    value = value.split(':')[1].strip()
    if value == 'NW' or value == 'MONW':
        return 0
    elif value == 'OW/OB':
        return 1
    else:
        return None

def convert_age(value):
    # Extract the value after the colon
    value = value.split(':')[1].strip()
    try:
        age = float(value)
        # Ensure the age is within a plausible adult range
        if 18 <= age <= 45:
            return age
        else:
            return None
    except ValueError:
        return None

def convert_gender(value):
    # Extract the value after the colon
    value = value.split(':')[1].strip()
    if value.lower() == 'man':
        return 1
    elif value.lower() == 'woman':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE181339', './preprocessed/Obesity/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical feature data
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Obesity', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Obesity/trait_data/GSE181339.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM5494930': [0.0, 21.0, 1.0], 'GSM5494931': [1.0, 23.0, 1.0], 'GSM5494932': [0, None, 0], 'GSM5494933': [1, None, 0], 'GSM5494934': [0, None, 0], 'GSM5494935': [1, None, 1], 'GSM5494936': [1.0, 18.0, 0.0], 'GSM5494937': [1, None, 0], 'GSM5494938': [0, None, 0], 'GSM5494939': [1.0, 23.0, 1.0], 'GSM5494940': [1.0, 23.0, 1.0], 'GSM5494941': [0, None, 1], 'GSM5494942': [1, None, 0], 'GSM5494943': [1, None, 0], 'GSM5494944': [1, None, 0], 'GSM5494945': [0, None, 0], 'GSM5494946': [0, None, 0], 'GSM5494947': [0, None, 0], 'GSM5494948': [1.0, 21.0, 1.0], 'GSM5494949': [1, None, 1], 'GSM5494950': [1.0, 26.0, 0.0], 'GSM5494951': [1, None, 1], 'GSM5494952': [0.0, 21.0, 1.0], 'GSM5494953': [1.0, 26.0, 0.0], 'GSM5494954': [0, None, 0], 'GSM5494955': [1, None, 1], 'GSM5494956': [1, None, 1], 'GSM5494957': [1, None, 0], 'GSM5494958': [1, None, 1], 'GSM5494959': [0, None, 1], 'GSM5494960': [0, None, 1], 'GSM5494961': [1, None, 0], 'GSM5494962': [1.0, 26.0, 0.0], 'GSM5494963': [0, None, 0], 'GSM549

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7', '8', '15', '18', '20', '21', '24', '25', '29', '32', '39', '41',
       '42', '44', '45', '46', '48', '51', '53', '55'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, nan, 'NM_001105533'], 'GB_ACC': [nan, nan, nan, nan, 'NM_001105533'], 'LOCUSLINK_ID': [nan, nan, nan, nan, 79974.0], 'GENE_SYMBOL': [nan, nan, nan, nan, 'CPED1'], 'GENE_NAME': [nan, nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1'], 'UNIGENE_ID': [nan, nan, nan, nan, 'Hs.189652'], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'unmapped', 'chr7:120901888-120901947'], 'CYTOBAND': [nan, nan, na

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the keys for identifier and gene symbol based on the annotation data
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print("Mapped gene data preview:")
print(preview_df(gene_data))


Mapped gene data preview:
{'GSM5494930': [9.356389, 6.588705, 10.0865905, 6.087023, 8.855058], 'GSM5494931': [9.580217, 6.861172, 8.5899135, 5.95844, 8.172307], 'GSM5494932': [9.920784, 7.055549, 9.4676615, 6.690681, 8.768802], 'GSM5494933': [9.504974, 6.792186, 7.930585000000001, 5.814862, 8.708854], 'GSM5494934': [9.533504, 7.192053, 9.596064, 5.822462, 8.534389], 'GSM5494935': [9.926714, 7.000017, 9.4711555, 5.521768, 8.529483], 'GSM5494936': [10.22561, 7.219546, 8.926901, 5.832344, 8.113828], 'GSM5494937': [9.708488, 6.974349, 9.7555435, 5.259415, 8.449762], 'GSM5494938': [9.759847, 7.343875, 8.151971, 6.574513, 8.988748], 'GSM5494939': [9.47079, 6.878397, 9.04759, 6.160754, 8.586938], 'GSM5494940': [9.301762, 7.038205, 8.9872485, 6.521225, 8.397392], 'GSM5494941': [9.486415, 7.187312, 8.189250000000001, 6.461885, 8.413836], 'GSM5494942': [9.778403, 7.21115, 8.521418, 6.238289, 8.581843], 'GSM5494943': [9.639646, 7.304646, 8.0447855, 6.407465, 8.790874], 'GSM5494944': [9.851406, 7.

### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Obesity/gene_data/GSE181339.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Obesity')

# 4. Save the cohort information.
save_cohort_info('GSE181339', './preprocessed/Obesity/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Obesity/GSE181339.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Obesity', the least common label is '0.0' with 4 occurrences. This represents 16.67% of the dataset.
The distribution of the feature 'Obesity' in this dataset is fine.

Quartiles for 'Age':
  25%: 20.5
  50% (Median): 22.0
  75%: 26.0
Min: 18.0
Max: 30.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 8 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

